In [ ]:
%load_ext autoreload
%autoreload 2
import tensorflow as tf
from pathlib import Path
from watch_recognition.reports import run_on_image_debug
from watch_recognition.models import convert_outputs_to_keypoints

from watch_recognition.reports import run_on_image_debug

In [ ]:
image_size = (224, 224)
model_path = "./models/keypoint/efficientnetb0/run_1632579369.409768.h5"
loaded_model = tf.keras.models.load_model(model_path, compile=False)
loaded_model.summary()

In [ ]:
for path in [
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.14.jpg"),
    Path("../example_data/Zrzut ekranu 2021-08-25 o 22.24.24.jpg"),
    Path("../example_data/test-image-2.jpg"),
]:
    test_image = tf.keras.preprocessing.image.load_img(
        path, "rgb", target_size=image_size, interpolation="bicubic",
    )
    test_image_np = tf.keras.preprocessing.image.img_to_array(test_image)

    run_on_image_debug(loaded_model, test_image_np)

In [ ]:
from watch_recognition.data_preprocessing import load_keypoints_data_as_kp
ROOT_DIR = ".."
X, y, filenames = load_keypoints_data_as_kp(
    Path("%s/download_data/keypoints/train" % ROOT_DIR),
)
X.shape, y.shape

In [ ]:
predicted = loaded_model.predict(X)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

In [ ]:
predicted.shape

In [ ]:
from watch_recognition.reports import calculate_distances_between_points

In [ ]:
(
    center_distances,
    hour_distances,
    minute_distances,
    top_distances,
) = calculate_distances_between_points(X, predicted, y)

In [ ]:
import pandas as pd

In [ ]:
from watch_recognition.augmentations import encode_keypoints_to_mask_np

In [ ]:
len(top_distances)

In [ ]:
df = pd.DataFrame({
    "center_distances":center_distances,
    "hour_distances":hour_distances,
    "minute_distances":minute_distances,
    "top_distances":top_distances,
    "filename": filenames,
})

In [ ]:
df['mean_distances'] = df.mean(axis=1).values

In [ ]:
df.head()

In [ ]:
df.sort_values("mean_distances").head(5)



In [ ]:
df.mean(axis=0)

In [ ]:
for i, data in df.sort_values("mean_distances", ascending=False).head(10).iterrows():
    scale_factor = X.shape[1] / predicted.shape[1]
    print(f"example {filenames[i]}")
    print(data)

    center_hat, hour_hat, minute_hat, top_hat = convert_outputs_to_keypoints(
        predicted[i]
    )
    center_hat, hour_hat, minute_hat, top_hat = [
        np.array(p.as_coordinates_tuple).astype(float)
        for p in (center_hat, hour_hat, minute_hat, top_hat)
    ]
    center_hat = center_hat * scale_factor
    top_hat = top_hat * scale_factor
    hour_hat = hour_hat * scale_factor
    minute_hat = minute_hat * scale_factor
    predicted_points = np.stack(
        [
            center_hat,
            top_hat,
            minute_hat,
            hour_hat,
        ]
    )
   
    rounded_predicted_points = np.round(predicted_points)
    print("targets|predictions")
    print(np.hstack((y[i][:, :2], rounded_predicted_points)))
    markers = ['.', '^', '*', 's']
    names = ['center', 'top', 'minute', 'hour']
    for point, marker, name in zip(predicted_points, markers, names):
        plt.scatter(*point, marker=marker, color='red', label=name)
    for point, marker, name in zip(y[i][:, :2], markers, names):
        plt.scatter(*point, marker=marker, color='green', label=name)

    plt.imshow(X[i])
    ax = plt.gca()
    # https://stackoverflow.com/a/4701285/8814045
    # Shrink current axis's height by 10% on the bottom
    box = ax.get_position()
    ax.set_position([box.x0, box.y0 + box.height * 0.1,
                 box.width, box.height * 0.9])

    # Put a legend below current axis
    ax.legend(loc='upper center', bbox_to_anchor=(0.5, -0.05),
          fancybox=True, shadow=True, ncol=5)
    plt.show()
    run_on_image_debug(loaded_model, X[i])
    print("#" * 100)